1. Посчитайте, сколько компаний закрылось.

In [ ]:
SELECT COUNT(id)
FROM company
WHERE status = 'closed';

2.
Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total .

In [ ]:
SELECT 
       funding_total 
FROM company
WHERE category_code = 'news'
    AND country_code = 'USA'
ORDER BY funding_total DESC;

3. Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно

In [ ]:
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code = 'cash'
    AND EXTRACT(YEAR FROM CAST(acquired_at AS DATE)) BETWEEN 2011 AND 2013;

4. Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на 'Silver'.

In [ ]:
SELECT first_name,
       last_name,
       twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%';

5. Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку 'money', а фамилия начинается на 'K'.

In [ ]:
SELECT *
FROM people
WHERE twitter_username LIKE '%money%'
      AND last_name LIKE 'K%';

6. Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

In [ ]:
SELECT country_code,
       SUM(funding_total)
FROM company 
GROUP BY country_code
ORDER BY SUM(funding_total) DESC;

7. Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

In [ ]:
SELECT funded_at,
       MIN(raised_amount),
       MAX(raised_amount)
FROM funding_round
GROUP BY funded_at
HAVING MIN(raised_amount) > 0
      AND MIN(raised_amount) <> MAX(raised_amount);

8. Создайте поле с категориями:
Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.
Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.
Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.
Отобразите все поля таблицы fund и новое поле с категориями.

In [ ]:
SELECT *,    
    CASE 
 WHEN invested_companies > 100 THEN 'high_activity'
 WHEN invested_companies < 20 THEN 'low_activity'
 ELSE 'middle_activity' 
 
END
FROM fund;   

9. Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [ ]:
SELECT  CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity,
       ROUND(AVG(investment_rounds))
FROM fund
GROUP BY activity
ORDER BY ROUND(AVG(investment_rounds));

10. Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы. 
Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. Выгрузите десять самых активных стран-инвесторов.
Отсортируйте таблицу по среднему количеству компаний от большего к меньшему, а затем по коду страны в лексикографическом порядке.

In [ ]:
SELECT country_code,
       MIN(invested_companies),
       MAX(invested_companies),
       AVG(invested_companies)
FROM fund
WHERE EXTRACT(YEAR FROM CAST(founded_at AS DATE)) BETWEEN 2010 AND 2012
GROUP BY country_code
HAVING MIN(invested_companies) > 0
ORDER BY AVG(invested_companies) DESC, country_code
LIMIT 10;

11. Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

In [ ]:
SELECT p.first_name,
       p.last_name,
       e.instituition
FROM people AS p  
LEFT OUTER JOIN education AS e ON p.id = e.person_id;

12. Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

In [ ]:
WITH a AS (SELECT p.first_name,
           p.last_name,
           e.instituition,
           c.name
    FROM people AS p  
    LEFT OUTER JOIN education AS e ON p.id = e.person_id
    LEFT OUTER JOIN company AS c ON p.company_id = c.id)
SELECT a.name,
       COUNT(DISTINCT a.instituition)
FROM a
WHERE a.name IS NOT NULL
GROUP BY a.name
ORDER BY COUNT(DISTINCT a.instituition) DESC
LIMIT 5;

13. Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

In [ ]:
SELECT DISTINCT c.name
FROM company AS c
WHERE c.status = 'closed' 
    AND c.id IN 
    (SELECT company_id
    FROM funding_round
    WHERE is_first_round = 1
         AND is_last_round = 1)

   

14. Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

In [ ]:
SELECT id
FROM people
WHERE company_id IN (SELECT DISTINCT c.id
        FROM company AS c
        WHERE c.status = 'closed' 
            AND c.id IN 
            (SELECT company_id
            FROM funding_round
            WHERE is_first_round = 1
                 AND is_last_round = 1));

15. Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [ ]:
WITH a AS (SELECT id
    FROM people
    WHERE company_id IN (SELECT DISTINCT c.id
        FROM company AS c
        WHERE c.status = 'closed' 
            AND c.id IN 
            (SELECT company_id
            FROM funding_round
            WHERE is_first_round = 1
                 AND is_last_round = 1))),
b AS (SELECT DISTINCT person_id AS id,
               instituition
      FROM education)
SELECT a.id,
       b.instituition
FROM a JOIN b ON a.id = b.id
ORDER BY a.id;       

16. Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

In [ ]:
WITH a AS (SELECT id
    FROM people
    WHERE company_id IN (SELECT DISTINCT c.id
        FROM company AS c
        WHERE c.status = 'closed' 
            AND c.id IN 
            (SELECT company_id
            FROM funding_round
            WHERE is_first_round = 1
                 AND is_last_round = 1))),
b AS (SELECT person_id AS id,
               instituition
      FROM education)
SELECT a.id,
       COUNT(b.instituition)
FROM a JOIN b ON a.id = b.id
GROUP BY a.id;       
                       

17. Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

In [ ]:
WITH a AS (SELECT p.id,
       COUNT(e.instituition)
FROM people AS p
JOIN education AS e ON p.id=e.person_id
WHERE p.company_id IN (SELECT id
                       FROM company
                       WHERE status='closed'
                       AND id IN (SELECT company_id
                              FROM funding_round
                              WHERE is_first_round=1 AND is_last_round=1))
GROUP BY p.id)
SELECT AVG(count)
FROM a;
               

18. Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook*.
*(сервис, запрещённый на территории РФ)

In [ ]:
WITH facebook AS (SELECT p.id,
                           COUNT(e.instituition)
                    FROM people AS p
                    INNER JOIN education AS e 
                    ON p.id=e.person_id
                    WHERE p.company_id IN (
                                        SELECT id
                                        FROM company
                                        WHERE name='Facebook'
                                          )
                    GROUP BY p.id)
 
SELECT AVG(count)
FROM facebook;

19. Составьте таблицу из полей:
name_of_fund — название фонда;
name_of_company — название компании;
amount — сумма инвестиций, которую привлекла компания в раунде.
В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [ ]:
SELECT f.name AS name_of_fund,
       c.name AS name_of_company,
       fr.raised_amount AS amount
FROM investment AS i
LEFT JOIN company AS c ON i.company_id=c.id
LEFT JOIN fund AS f ON f.id=i.fund_id
LEFT JOIN funding_round AS fr ON i.funding_round_id = fr.id
WHERE c.milestones > 6
AND EXTRACT(YEAR FROM CAST(funded_at AS date)) BETWEEN 2012 AND 2013        

20. Выгрузите таблицу, в которой будут такие поля:
название компании-покупателя;
сумма сделки;
название компании, которую купили;
сумма инвестиций, вложенных в купленную компанию;
доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.
Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы. 
Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

In [ ]:
SELECT cmp.name AS name_acquiring_company ,
       acq.price_amount,
       com.name AS name_acquired_company,
       com.funding_total,
       ROUND(acq.price_amount/com.funding_total) AS percent
FROM acquisition AS acq
LEFT JOIN company AS cmp ON acq.acquiring_company_id=cmp.id
LEFT JOIN company AS com ON acq.acquired_company_id=com.id
WHERE acq.price_amount > 0
    AND com.funding_total>0
ORDER BY acq.price_amount DESC
LIMIT 10;

21.
Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

In [ ]:
SELECT c.name,
       EXTRACT(MONTH FROM CAST(fr.funded_at AS date)) AS month
FROM company AS c
LEFT JOIN funding_round AS fr ON c.id=fr.company_id
WHERE c.category_code = 'social'
      AND fr.raised_amount > 0
      AND EXTRACT(YEAR FROM CAST(fr.funded_at AS date)) BETWEEN 2010 AND 2013;

22.
Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
номер месяца, в котором проходили раунды;
количество уникальных названий фондов из США, которые инвестировали в этом месяце;
количество компаний, купленных за этот месяц;
общая сумма сделок по покупкам в этом месяце.

In [ ]:
WITH a AS (SELECT EXTRACT(MONTH FROM CAST(fr.funded_at AS date)) AS month_number,
                      COUNT(DISTINCT f.name) AS fund_count_USA       
               FROM funding_round AS fr
               LEFT OUTER JOIN investment AS i ON fr.id=i.funding_round_id
               LEFT OUTER JOIN fund AS f ON i.fund_id=f.id
               WHERE f.country_code='USA'
               AND EXTRACT(YEAR FROM CAST(fr.funded_at AS date)) BETWEEN 2010 AND 2013
               GROUP BY  EXTRACT(MONTH FROM CAST(fr.funded_at AS date))),
                         
b  AS (SELECT EXTRACT(MONTH FROM CAST(acquired_at AS date)) AS month_number,
                      COUNT(id) AS company_count,
                      SUM(price_amount) AS total_amount
               FROM  acquisition
               WHERE EXTRACT(YEAR FROM CAST(acquired_at AS date)) BETWEEN 2010 AND 2013
               GROUP BY EXTRACT(MONTH FROM CAST(acquired_at AS date)))
SELECT a.month_number,
       a.fund_count_usa,
       b.company_count,
       b.total_amount
FROM a       
LEFT JOIN b ON a.month_number=b.month_number;
       

23.
Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
WITH a AS (SELECT country_code,
           AVG(funding_total) AS avg_2011
           FROM company
           WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2011
           GROUP BY country_code),
b AS (SELECT country_code,
           AVG(funding_total) AS avg_2012
           FROM company
           WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2012
           GROUP BY country_code),  
c AS (SELECT country_code,
           AVG(funding_total) AS avg_2013
           FROM company
           WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2013
           GROUP BY country_code)     
SELECT a.country_code,
       avg_2011,
       avg_2012,
       avg_2013
FROM a
JOIN b ON a.country_code=b.country_code
JOIN c ON c.country_code=b.country_code
ORDER BY avg_2011 DESC;